## Better Way 40
### super로 부모 클래스를 초기화하라

#### 자식클래스에서 부모클래스를 초기화하는 오래된 방법
: 자식클래스에서 부모 클래스의 __init__메서드 직접 호출 

In [5]:
class MyBaseClass:
    def __init__(self, value):
        self.value = value

class MyChildClass(MyBaseClass):
    def __init__(self):
        MyBaseClass.__init__(self, 5)

#### 기본적인 클래스 계층에서는 잘 작동하지만 다른 경우에서는 잘못될 수 있다,


### 1. 클래스가 다중 상속에 의해 영향을 받은 경우
상위 클래스의 \__init\__메서드를 직접호출하면 프로그램이 예측할 수 없는 방향으로 작동할 수 있다,

**문제1. 모든 하위클래스에서 __init__의 호출순서가 정해져 있지 않다.**

In [6]:
# 인스턴스의 value필드에 대해 작용하는 두 상위 클래스를 정의하는 코드
class TimesTwo:
    def __init__(self):
        self.value *= 2
        
class PlusFive:
    def __init__(self):
        self.value += 5

In [7]:
# 부모 클래스를 TimesTwo, PlusFive순서로 정의하는 코드
class OneWay(MyBaseClass, TimesTwo, PlusFive):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)
        
# 위 클래스의 인스턴스를 만들면 부모 클래스의 순서에 따라 초기화가 실행된다.
foo = OneWay(5)
print('첫 번째 부모 클래스 순서에 따른 값은 (5*2) + 5 =', foo.value)

첫 번째 부모 클래스 순서에 따른 값은 (5*2) + 5 = 15


In [8]:
# 같은 부모클래스를 사용하지만 부모클래스를 나열한 순서가 다른 경우
class AnotherWay(MyBaseClass, PlusFive, TimesTwo):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)   
        # 부모 클래스의 생성자를 호출하는 순서는 그대로 이므로 결과는 같다 
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

In [9]:
bar = AnotherWay(5)
print('두 번째 부모 클래스 순서에 따른 값은', bar.value)

두 번째 부모 클래스 순서에 따른 값은 15


#### 문제 2. 클래스 정의에 나열한 부모 클래스의 순서와 부모 생성자를 호출한 순서가 일치 하지 않는다. 
이 문제는 발견이 어렵고, 코드를 처음보고 이해하기도 어렵다.

<br>

### 문제 2. 다이아몬드 상속인 경우
**다이아몬드 상속:** 두 가지 서로 다른 클래스를 상속할 때, 그 두 클래스가 같은 조상 클래스를 가진 경우.

#### 공통 조상 클래스의 __init__메서드가 여러번 호출될 수 있어 예기치 못한 방향으로 작동가능.


In [11]:
# MyBaseClass를 상속하는 두 가지 자식 클래스 정의
class TimesSeven(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value *= 7
        
class PlusNine(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value += 9
          

In [12]:
# 위 두 클래스를 모두 상속하는 자식 클래스 정의
class ThisWay(TimesSeven, PlusNine):
    def __init__(self, value):
        TimesSeven.__init__(self, value)
        PlusNine.__init__(self, value)
        
foo = ThisWay(5)
print('(5 *  7) + 9 = 44가 나와야 하지만 실제로는', foo.value)

(5 *  7) + 9 = 44가 나와야 하지만 실제로는 14


1. TimesSeven.\__init\__호출 -> self.value(5) *= 7 -> self.value = 35 
2. PlusNine.\__init\__을 호출 -> self.value(5) += 9 -> self.value = 14
### 2.에서 self.value의 값이 다시 5로 초기화된다!
프로그래머를 당황하게 하고, 복잡한 코드의 경우 디버깅이 힘들다.

<br>

### 해결책: ```super``` 내장함수와 MRO

1. **```super```:** 다이아몬드 계층의 공통 상위클래스를 단 한 번만 호출하도록 보장.
2. **MRO(Method Resolution Order):** C3 선형화(linearization) 알고리즘 사용해 상위 클래스를 초기화하는 순서를 정의.

In [13]:
# 다이아몬드 모양의 클래스 구조를 다시 만들고, super를 사용해 부모클래스 초기화
class TimesSevenCorrect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)      #MyBaseClass.__init__이 단 한 번만 실행된다. 
        self.value *= 7

class PlusNineCorrect(MyBaseClass):
    def __init__(self, value):     
        super().__init__(value)      #MyBaseClass.__init__이 단 한 번만 실행된다. 
        self.value += 9

In [14]:
class GoodWay(TimesSevenCorrect, PlusNineCorrect): # 호출할 때 파라미터로 지정한 순서가 아니라
    def __init__(self, value):                     # 이 클래스에 대한 MRO 정의를 따른다.
        super().__init__(value)
        
foo = GoodWay(5)
print('7 * (5 + 9) = 98이 나와야 하고 실제로도', foo.value)

7 * (5 + 9) = 98이 나와야 하고 실제로도 98


In [15]:
#mro메서드를 사용해 MRO 순서를 확인하기
mro_str = '\n'.join(repr(cls) for cls in GoodWay.mro())
print(mro_str)

<class '__main__.GoodWay'>
<class '__main__.TimesSevenCorrect'>
<class '__main__.PlusNineCorrect'>
<class '__main__.MyBaseClass'>
<class 'object'>


위 순서로  **각 클래스의 \__init\__을 호출하고 상속 다이아몬드의 정점에 도달한 후**   
각 초기화 메서드는 **각 클래스의 \__init\__이 호출된 역순으로 작업 수행**

Ex)   
MyBaseClass.\__init\은 value에 5대입   
PlusNineCorrect.\__init\__은 value에 9를 더해서 14로 만듬   
TimesSevenCorrect.\__init\__d은 value에 7을 곱해서 98로 만듬  

<br>

### super().__init__ 호출의 장점 
### 1. 다중 상속을 튼튼하게 하고, 유지보수를 더 편하게 해준다.
* MyBaseClass 이름 바꾸기
* TimesSevenCorrec/PlusNineCorrect가 상속받는 상위 클래스 변경   
### 각각의 \__init\__메서드 정의를 바꿀 필요 없다.

### 2. super 함수에 두 가지 파라미터 넘길 수 있다.
1. 접근하고 싶은 MRO 뷰를 제공할 부모타입
2. 1.에 지정한 타입의 MRO 뷰에 접근할 때 사용할 인스턴스

In [16]:
# super 함수에 파라미터 사용 예
class ExplicitTrisect(MyBaseClass):
    def __init__(self, value):
             #MRO뷰 부모타입   # 파라미터1에 접근할 인스턴스
        super(ExplicitTrisect, self).__init__(value)
        self.value /= 3

#### object 인스턴스 초기화시에는 두 파라미터 지정 불필요   
-> 객체지향에서 모든 클래스의 공통 조상인 object를 말하는 것 같은데... 어떻게 생각하시나요?
   
### 클래스 정의에서 아무 인자도 지정하지 않고 super 호출   
### -> 파이썬 컴파일러가 자동으로 올바른 파라미터(\__class\__, self)넣어줌

In [3]:
# 위의 ExplicitTrisect와 아래의 두 가지 사용법은 모두 동일
class AutomaticTrisect(MyBaseClass):
    def __init__(self, value):
        super(__class__, self).__init__(value)
        self.value /= 3
        
class ImplicitTrisect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)
        self.value /= 3
        
# assertion 에러가 발생하지 않는다.        
assert ExplicitTrisect(9).value == 3
assert AutomaticTrisect(9).value == 3
assert ImplicitTrisect(9).value == 3

NameError: name 'ExplicitTrisect' is not defined

### super에 파라미터 전달해야 하는 경우
**자식 클래스에서 상위 클래스의 특정 기능에 접근해야 하는 경우 뿐!**   
ex) 특정 기능 감싸기, 재사용

<br>

### 요약
* 파이썬은 **표준 메서드 결정순서(MRO)**를 활용해 **상위 클래스 초기화 순서**와 **다이아몬드 상속 문제** 해결
* **부모클래스 초기화 할 때 ```super()```를 사용하라.** 파이썬 컴파일러가 올바른 파라미터를 자동으로 넣어준다.